# NLPipe - Demo (simple)
<p>#TODO: Needs description of what this file is</p>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mvmagni/magritte/blob/main/demo_colab/NLPipe_demo_simple_colab.ipynb)

In [ ]:
# Remove existing magritte repo if present
!rm -rf magritte

# Clone repo for utilities
!git clone https://github.com/mvmagni/magritte

In [ ]:
# Install and import required packages for data.
!pip install -r magritte/requirements.txt --quiet

# Install any requirements specific to the demo
!pip install -r magritte/demo_colab/requirements_demo.txt --quiet


from cleantext import clean

# Initialize environment

In [ ]:
import sys
import os

PROJECT_DIR = f'{os.getcwd()}/magritte'
DEMO_DATA_DIR = f'{PROJECT_DIR}/data'
DEMO_DATA_FILE = f'{DEMO_DATA_DIR}/IMDB Dataset.csv.gz'
UTILITIES_DIR = f'{PROJECT_DIR}/utilities'

# Add the UTILITY_DIR to the path to import files
sys.path.append(UTILITIES_DIR)

In [ ]:
import pandas as pd
import numpy as np

# Load Data
<p>Loading the IMDB Dataset<br>
50,000 records on pos/neg sentiment analysis<br>
25,000 records per class<br>

In [ ]:
# Loading the IMDB Dataset
ORIG_DF = pd.read_csv(DEMO_DATA_FILE, compression='gzip')

# Changing target column from string to 0,1. Label encoding not yet included in framework
ORIG_DF['sentiment'] = np.where(ORIG_DF['sentiment']=='positive', 1, 0)

print(f'Data shape: {ORIG_DF.shape}')
display(ORIG_DF.head())

In [ ]:
# Setting up variables for later usage in DataPackage
DATA_COLUMN = 'review'
TARGET_COLUMN = 'sentiment'
UNIQUE_COLUMN = None   # Unique index column. If None, one will be created

# DataPackageParams
<p>#TODO: Needs description of what DataPackageParams is</p>

In [ ]:
# DataPackageParams are optional. Can do it manually or store and save the params for re-run
from DataPackage import DataPackageParams
myDPP = DataPackageParams(
                    process_params=False, # True=run all data cleanup/setup on load
                    
                    # Class Balance
                    sample_size = None, # Can be set to an absolute value. None means undersample to smallest
                                        
                    # Text Cleaning Params
                    fix_unicode=True,  # fix various unicode errors
                    to_ascii=True,  # transliterate to closest ASCII representation
                    lower=True,  # lowercase text
                    no_line_breaks=False,  # fully strip line breaks as opposed to only normalizing them
                    no_urls=False,  # replace all URLs with a special token
                    no_emails=False,  # replace all email addresses with a special token
                    no_phone_numbers=False,  # replace all phone numbers with a special token
                    no_numbers=True,  # replace all numbers with a special token
                    no_digits=False,  # replace all digits with a special token
                    no_currency_symbols=False,  # replace all currency symbols with a special token
                    no_punct=False,  # remove punctuations
                    replace_with_punct="",  # instead of removing punctuations you may replace them
                    replace_with_url="<URL>",
                    replace_with_email="<EMAIL>",
                    replace_with_phone_number="<PHONE>",
                    replace_with_number="",
                    replace_with_digit="0",
                    replace_with_currency_symbol="<CUR>",
                    lang="en",  # set to 'de' for German special handling

                    # Remove stopwords
                    remove_stopwords=True, # Removes stopwords
                    stopword_language='english',
        
                    # train test split params
                    stratifyColumn=None, # If None will be autoset to target_column in DataPackage
                    train_size=0.8, # Can be percent or absolute number
                    random_state=765,
                    shuffle=True,

                    # Encoding params
                    encoding_type='TFIDF', # Currently only supports TFIDF encoding, TBA: BERT, GLOVE, Word2Vec
                    max_features=100 # Currently only used in TFIDF
                    )

# Data Package
<p>#TODO: Needs description of what this file is</p>

In [ ]:
# Create the DataPackage
from DataPackage import DataPackage
myDP = DataPackage(original_data = ORIG_DF,
                   data_column = DATA_COLUMN,
                   target_column = TARGET_COLUMN,
                   unique_column = UNIQUE_COLUMN,
                   data_package_params = myDPP)

# Experiment Manager

In [ ]:
from ExperimentManager import ExperimentManager

In [ ]:
# Create the classifier used for the first Experiment
from xgboost import XGBClassifier
classifier1 = XGBClassifier(eval_metric='mlogloss', 
                            tree_method='gpu_hist',
                            use_label_encoder=False,
                            max_depth=5,
                            n_estimators=100)

In [ ]:
# Create the Experiment Manager 
myEM = ExperimentManager(project_name='NLPipe',
                         experiment_name='XGB depth:5 est:100',
                         classifier=classifier1,
                         data_package=myDP)

                         

In [ ]:
# Add another experiment
from sklearn.ensemble import RandomForestClassifier

classifier2 = RandomForestClassifier(n_jobs=-1,
                                     max_depth=10,
                                     n_estimators=10)

myEM.add_experiment(experiment_name='RF depth:10 est:10',
                    classifier=classifier2)

In [ ]:
# Add another experiment
classifier3 = XGBClassifier(eval_metric='mlogloss', 
                           tree_method='gpu_hist',
                           use_label_encoder=False,
                           max_depth=2,
                           n_estimators=20)

myEM.add_experiment(experiment_name='XGB depth:2 est:20',
                    classifier=classifier3)

In [ ]:
# Add another experiment
classifier4 = RandomForestClassifier(n_jobs=-1,
                                     max_depth=2,
                                     n_estimators=5)

myEM.add_experiment(experiment_name='RF depth:2 est:5',
                    classifier=classifier4)

# ExperimentManager - run_experiment

In [ ]:
myEM.list_experiments()

In [ ]:
axis_labels = [0,1] #This won't be needed once the label encoder has been included
myEM.run_experiment(axis_labels=axis_labels,
                    n_jobs=-1,  # -1 means use all available processors, otherwise include number
                    index=None) # index=None means process all experiements, 
                                # otherwise provide index of single experiment to run

# Experiment Summary

In [ ]:
myEM.display_experiment_summary(index=0, axisLabels=axis_labels)

In [ ]:
#myEM.display_experiment_summary(index=1, axisLabels=axis_labels)

# SHAP

In [ ]:
myEM.show_shap_waterfall(model_index=0,
                         value_index=0)

In [ ]:
myEM.show_shap_waterfall(model_index=2,
                         value_index=0)

In [ ]:
myEM.show_shap_summary(model_index=0)

# Persistence - Experiment Manager

In [ ]:
filename_EM = f'{PROJECT_DIR}/testExperimentManager'
myEM.save(filename_EM)

In [ ]:
recoveredEM = ExperimentManager.load(filename_EM)

In [ ]:
recoveredEM.display()

# Scratchpad

In [ ]:
!pip freeze > requirements.txt